In [2]:
import sympy as sp 
from sympy import Derivative
from IPython.display import display, Math

from libs.display import *
from libs.utils_sympy import *
from libs.reconstruction_amr import * 
DEBUG = 0
ORDER_SERIES = 4 # ordre des développements limités. 
DT_FROM_CFL = 0 # choisir si on défini dt à partir de la CFL ou si l'on défini la CFL à partir de dt. 
CAUCHY_KOVALESKAYA = 1
D = sp.Symbol("D", real=True) #Coefficient de diffusion
x, t = sp.symbols("x t", real=True)#Varaiables espace-temps
dx = sp.Symbol(r"\Delta x", real=True)#Pas de discrétisation spatial (niveau fin et grille uniforme)
u = sp.Function("u")#Fonction "régulière" vérifiant le schéma

if DT_FROM_CFL :
    CFL_DIFF = sp.Symbol(r"\lambda", real=True)
    dt = dx**2 * CFL_DIFF / D
else : 
    dt = sp.Symbol(r"\Delta t", real=True)
    CFL_DIFF = D * dt / dx**2
CFL_DIFF = D * dt / dx**2 

In [3]:
u_xplus1 = developpement_taylor_spatial(u,+dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x+dx
u_xplus2 = developpement_taylor_spatial(u,+2*dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x+2dx
u_xmoins1 = developpement_taylor_spatial(u,-dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x-dx
u_xmoins2 = developpement_taylor_spatial(u,-2*dx,ORDER_SERIES,x,t) # développement de Taylor en espace en x-2dx
reste_serie_en_temps =((developpement_taylor_temps(u,dt,ORDER_SERIES,x,t) - u(x,t))/dt - sp.diff(u(x,t),t)).expand() # Erreur résiduelle lorsque l'on approxime du/dt = (u^t+1 - u^t)/dt
derivee_temporelle = sp.diff(u(x, t), t) # dérivée temporelle d'ordre 1


In [ ]:

# Méthode des lignes RK1
print("Eq modifée temps-espace RKE1")
flux_total_sans_AMR_RK1 = (CFL_DIFF * 
                           (u_xplus1 - 2*u(x,t) + u_xmoins1)).expand()
u_t_RK1 = - reste_serie_en_temps + 1/dt*flux_total_sans_AMR_RK1
u_t_RK1 = u_t_RK1.expand()
u_t_RK1 = apply_cauchy_kovaleskaya(u_t_RK1,u,x,t,D,ORDER_SERIES)
eq_sans_AMR_RK1 = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t_RK1)}"
display(Math(eq_sans_AMR_RK1))

# Méthode des lignes RK2
print("Eq modifée temps-espace RKE2")
flux_total_sans_AMR = CFL_DIFF*(
                       (u_xplus1 - 2*u(x,t) + u_xmoins1) 
                       +sp.Rational(1,2) * CFL_DIFF*(u_xplus2 - 4*u_xplus1 + 6*u(x,t) - 4*u_xmoins1 + u_xmoins2)
                       ).expand()
u_t = - reste_serie_en_temps +(1/dt) * flux_total_sans_AMR # équation équivalente brute. 
u_t = u_t.expand()
u_t = apply_cauchy_kovaleskaya(u_t,u,x,t,D,ORDER_SERIES)
second_membre = rendreJoli(u_t,u,x,t,dx,dt,ORDER_SERIES)
eq_sans_AMR_Latex = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t)}"
display(Math(eq_sans_AMR_Latex))

### Christian 
print("Eq modifée temps-espace Version Christian")
flux_total_sans_AMR_Chr = CFL_DIFF*((u_xplus1 - 2*u(x,t) + u_xmoins1)
                           +sp.Rational(1,2)*(CFL_DIFF - sp.Rational(1,6))* (u_xplus2 - 4*u_xplus1 + 6*u(x,t) - 4*u_xmoins1 + u_xmoins2)).expand()
u_t_Chr = - reste_serie_en_temps + 1/dt*flux_total_sans_AMR_Chr
u_t_Chr = u_t_Chr.expand()
u_t_Chr = apply_cauchy_kovaleskaya(u_t_Chr,u,x,t,D,ORDER_SERIES)
eq_sans_AMR_Chr = f"{sp.latex(derivee_temporelle)} = {sp.latex(u_t_Chr)}"
display(Math(eq_sans_AMR_Chr))

Eq modifée temps-espace RKE1


<IPython.core.display.Math object>

Eq modifée temps-espace RKE2


<IPython.core.display.Math object>

Eq modifée temps-espace Version Christian


<IPython.core.display.Math object>

# Introduction of MRA

In [5]:
s_hat = sp.Symbol('$\hat s$', positive=True)
deltal = sp.Symbol(u'\\Delta l', positive=True)
dictionnary             = computeEqModif(u,x,t,deltal,dx,dt,D,CFL_DIFF,ORDER_SERIES,CAUCHY_KOVALESKAYA=CAUCHY_KOVALESKAYA,stencil=1,DEBUG=DEBUG)
print("--------------------------------------  EQUA. MODIFIEES [RK1] --------------------------------------")
_,_,texRK1    = dictionnary["RK1"]
print("\n       *   SANS Multi-Résolution   ❌\n")
display(Math(eq_sans_AMR_Latex))
print("\n       *  AVEC Multi-Résolution    ✅")
display(Math(texRK1))

print("--------------------------------------  EQUA. MODIFIEES [RK2] --------------------------------------")
_,_,texRK2             = dictionnary["RK2"]
print("\n       *   SANS Multi-Résolution   ❌\n")
display(Math(eq_sans_AMR_Latex))
print("\n       *  AVEC Multi-Résolution   ✅")
display(Math(texRK2))

print("--------------------------------------  EQUA. MODIFIEES [RK2 - Christian] --------------------------------------")
_,_,texChristian             = dictionnary["Christian"]
print("\n       *   SANS Multi-Résolution   ❌\n")
display(Math(eq_sans_AMR_Latex))
print("\n       *  AVEC Multi-Résolution   ✅")
display(Math(texChristian))

--------------------------------------  EQUA. MODIFIEES [RK1] --------------------------------------

       *   SANS Multi-Résolution   ❌



<IPython.core.display.Math object>


       *  AVEC Multi-Résolution    ✅


<IPython.core.display.Math object>

--------------------------------------  EQUA. MODIFIEES [RK2] --------------------------------------

       *   SANS Multi-Résolution   ❌



...

<IPython.core.display.Math object>

--------------------------------------  EQUA. MODIFIEES [RK2 - Christian] --------------------------------------

       *   SANS Multi-Résolution   ❌



<IPython.core.display.Math object>


       *  AVEC Multi-Résolution   ✅


<IPython.core.display.Math object>